# Financial CSV Agent with Native Multi-Step Cohere API

In [1]:
import os
from typing import List

import numpy as np
import cohere
import langchain
import langchain_core
import langchain_experimental
import pandas as pd
from langchain.agents import Tool
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_experimental.utilities import PythonREPL
from dotenv import load_dotenv

In [2]:
# versions
print('cohere version:', cohere.__version__)
print('langchain version:', langchain.__version__)
print('langchain_core version:', langchain_core.__version__)
print('langchain_experimental version:', langchain_experimental.__version__)

cohere version: 5.6.2
langchain version: 0.2.11
langchain_core version: 0.2.24
langchain_experimental version: 0.0.63


### setup variables envi

In [3]:
load_dotenv()

True

In [4]:
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
CHAT_URL= "https://api.cohere.ai/v1/chat"
COHERE_MODEL = 'command-r-plus'
co = cohere.Client(api_key=COHERE_API_KEY)

In [5]:
os.getcwd()

'/Users/cindybtari/Documents/cmpwg_cohere/cmpwg/cmpwg_hackjakarta'

In [6]:
!ls

LICENSE               data                  pyproject.toml
README.md             financial-agent.ipynb requirements.txt
cohere_api_test.py    poetry.lock           src


In [7]:
df = pd.read_csv('data/gmr_metrics.csv')

In [8]:
import src.financial_calculator as fc

In [17]:
df = pd.read_csv('data/food_dataset_v1.csv')

In [18]:
weekly_report =fc.process_data('data/food_dataset_v1.csv')

In [20]:
weekly_report.tail()

,merchant_id,date,product,weekly_total_price,weekly_total_discount_price,weekly_total_discounts,merchant_name,merchant_area,category,display,description,total_orders,total_revenue,average_discount,most_ordered_item,least_ordered_item,most_ordered_item_category
682,M683,2024-07-28,"Crunchy Signature Kakao, Crunchy Dirty Mokao, ...",612000.0,25000.0,1,"XIBOBA, Tenggilis Surabaya",surabaya,Minuman,Kakao Series,Cadburry chocolate milk dipadukan dengan choco...,25,587000.0,0.040850,Crunchy Signature Kakao,Crunchy Signature Kakao,Brown Sugar Boba Milk Tea
683,M684,2024-07-28,Dirty Matcha Dalgona With Hokkaido Milk Puddin...,612000.0,25000.0,1,"XIBOBA, Tunjungan Plaza",surabaya,Minuman,Dalgona Matcha Series,Signature biscoff sauce dipadukan dengan susu ...,25,587000.0,0.040850,Dirty Matcha Dalgona With Hokkaido Milk Pudding,Dirty Matcha Dalgona With Hokkaido Milk Pudding,Brown Sugar Boba Milk Tea
684,M685,2024-07-28,Promo Opening Beli 2 Paket Yummy 1 Gratis 1 Ay...,221500.0,171500.0,11,"Yasaka Fried Chicken, Sukolegok",surabaya,Aneka nasi/Ayam & bebek/Cepat saji,Promo Opening,2 Paket Yummy 1 (Paha Bawah/Sayap + Nasi) Grat...,12,50000.0,0.774266,Promo Opening Beli 2 Paket Yummy 1 Gratis 1 Ay...,Promo Opening Beli 2 Paket Yummy 1 Gratis 1 Ay...,Promo Opening Cuman 10 Ribu Dapat 1 Ayam Crisp...
685,M686,2024-07-28,"ICE OVALTINE, ICE BUBLEGUM, ICE TEMUMANTAN, BA...",1240000.0,0.0,0,"Yakini, Krian",surabaya,Bakmie/Kopi/Minuman,ORIGINAL,Ice Creamy Choco Chocolate ( Gelas Jumbo ),99,1240000.0,0.000000,ICE OVALTINE,ICE OVALTINE,ICE GREEN TEA
686,M687,2024-07-28,"Black Oreo, Lemon Tea, Ice Cream Vanilla Caram...",2557550.0,0.0,0,"Zeger!, Sidoarjo",surabaya,Minuman/Kopi,Flavour Series,Nikmatnya Premium Black Oreo bikin kamu makin...,79,2557550.0,0.000000,Black Oreo,Black Oreo,Lemon Tea


In [22]:
df.head()

,merchant_id,date,product,weekly_total_price,weekly_total_discount_price,weekly_total_discounts,merchant_name,merchant_area,category,display,description,total_orders,total_revenue,average_discount,most_ordered_item,least_ordered_item
0,M001,2024-07-21,"Hot Millbro, Hot 330 Helateh, Hot Ilusi Kopi, ...",516500.0,0.0,0,"330 Kopi, Ciledug",jakarta,Kopi/Minuman/Roti,Signature,Sajian Susu Coklat Milo Plus Espresso Dengan R...,20,516500.0,0.0,Hot Millbro,Hot Millbro
1,M001,2024-07-28,"Ice Almara Kopi (kopi Susu Gula Aren), Ice Haz...",1022000.0,0.0,0,"330 Kopi, Ciledug",jakarta,Kopi/Minuman/Roti,Signature,Sajian Kopi Susu Gula Aren Yang Berbeda Dari K...,44,1022000.0,0.0,Ice Almara Kopi (kopi Susu Gula Aren),Ice Almara Kopi (kopi Susu Gula Aren)
2,M002,2024-07-21,"Chicken Steak Pops, Tenderloin Steak Beef Pop,...",482000.0,0.0,0,"Abe Steak, Cibubur",jakarta,Aneka nasi/Minuman/Barat,New Varian,"Chicken, Kentang, Mix Vegetable, Brown Sauce",19,482000.0,0.0,Chicken Steak Pops,Chicken Steak Pops
3,M002,2024-07-28,"Hemat Super 5, Rib Eye Import 175 Gram, Lemon ...",1294000.0,0.0,0,"Abe Steak, Cibubur",jakarta,Aneka nasi/Minuman/Barat,Paket Hemat,5 Chicken Steak + 5 Es Teh Manis,35,1294000.0,0.0,Hemat Super 5,Hemat Super 5
4,M003,2024-07-21,"Lychee Popping Boba, Mango Juice, Ice Age Melt...",2690600.0,0.0,0,"Abuba Steak, Cikarang",jakarta,Barat,New Menu,"Kandungan gizi dengan Gramasi 1 gelas, Total ...",33,2690600.0,0.0,Lychee Popping Boba,Mango Juice


In [21]:
text = (
    "Ice cream is a sweetened frozen food typically eaten as a snack or dessert. "
    "It may be made from milk or cream and is flavoured with a sweetener, "
    "either sugar or an alternative, and a spice, such as cocoa or vanilla, "
    "or with fruit such as strawberries or peaches. "
    "It can also be made by whisking a flavored cream base and liquid nitrogen together. "
    "Food coloring is sometimes added, in addition to stabilizers. "
    "The mixture is cooled below the freezing point of water and stirred to incorporate air spaces "
    "and to prevent detectable ice crystals from forming. The result is a smooth, "
    "semi-solid foam that is solid at very low temperatures (below 2 °C or 35 °F). "
    "It becomes more malleable as its temperature increases.\n\n"
    'The meaning of the name "ice cream" varies from one country to another. '
    'In some countries, such as the United States, "ice cream" applies only to a specific variety, '
    "and most governments regulate the commercial use of the various terms according to the "
    "relative quantities of the main ingredients, notably the amount of cream. "
    "Products that do not meet the criteria to be called ice cream are sometimes labelled "
    '"frozen dairy dessert" instead. In other countries, such as Italy and Argentina, '
    "one word is used fo\r all variants. Analogues made from dairy alternatives, "
    "such as goat's or sheep's milk, or milk substitutes "
    "(e.g., soy, cashew, coconut, almond milk or tofu), are available for those who are "
    "lactose intolerant, allergic to dairy protein or vegan."
)

response = co.summarize(
    text=text,
)
print(response)

id='9ea906eb-384b-44a3-95af-328acf0b3b7e' summary='Ice cream is a frozen dessert or snack usually made with cream or milk and flavoured with sweeteners, fruit or spices. It is semi-solid foam with ice crystals and air spaces. It becomes more malleable as its temperature increases. The meaning of the name "ice cream" varies and in some countries, such as Italy, it is used to refer to all variants. Dairy alternatives are available for those allergic to or intolerant of lactose.' meta=ApiMeta(api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None), billed_units=ApiMetaBilledUnits(input_tokens=337, output_tokens=89, search_units=None, classifications=None), tokens=None, warnings=None)


In [24]:
# only get summary from response

response.summary

'Ice cream is a frozen dessert or snack usually made with cream or milk and flavoured with sweeteners, fruit or spices. It is semi-solid foam with ice crystals and air spaces. It becomes more malleable as its temperature increases. The meaning of the name "ice cream" varies and in some countries, such as Italy, it is used to refer to all variants. Dairy alternatives are available for those allergic to or intolerant of lactose.'

In [25]:
df = pd.read_csv('data/reviews.csv')

In [26]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [27]:
df_review = pd.DataFrame()
df_review['rating'] = df['Score']
df_review['review'] = df['Text']

In [28]:
df_review.head()

,rating,review
0,5,I have bought several of the Vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...
2,4,This is a confection that has been around a fe...
3,2,If you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...


In [ ]:
num_merchants = 687
merchant_ids = [f'M{str(i).zfill(3)}' for i in range(1, num_merchants + 1)]

# Randomly assign Merchant_IDs to each review
np.random.seed(0)  # For reproducibility
data['Merchant_ID'] = np.random.choice(merchant_ids, size=len(data))